In [ ]:
from dispatch_jobs import DB, KEY_PREFIX
import pickle
import logging

logging.info(f'key prefix is {KEY_PREFIX}')
keys = DB.keys(KEY_PREFIX + '*')

In [ ]:
key = keys[0]

In [ ]:
info = DB.hgetall(key)
save_file = info['save_file']

with open(save_file, 'rb') as f:
    d = pickle.load(f)

pw_opposition_diagrams = d['pw_opposition_diagrams']  # list of 36 diagrams, each of shape (1, n, 3)
other_persistence_diagrams = d['other_persistence_diagrams']  # other_persistence_diagrams: list of 4 diagrams, each of shape (1, n, 3)

# For this first draft, just use one diagram

diagram = pw_opposition_diagrams[0]
